In [180]:
from gurobipy import Model, GRB
import polars as pl # Using Polars for educational purposes
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt

stocks = pl.read_csv("stocks.csv")

In [181]:
stocks.head()

Date,AAPL,GOOG,IBM,MARA,NVDA
str,f64,f64,f64,f64,f64
"""1/16/2023""",137.87,99.28,141.2,8.07,178.39
"""1/23/2023""",145.118851,100.709999,134.389999,8.02,203.649994
"""1/30/2023""",153.641205,105.220001,136.940002,7.07,211.0
"""2/6/2023""",150.170624,94.860001,135.600006,5.92,212.649994
"""2/13/2023""",151.933685,94.589996,135.020004,7.7,213.880005


In [182]:
# Get all the stock names to iterate over
stock_names = stocks.columns[1:]

# Calculate the weekly return percentage for each stock
stocks = (
    stocks.with_columns(
        ((pl.col(s) - pl.col(s).shift(1)) / pl.col(s).shift(1) * 100.0).alias(s)
        for s in stock_names
    )
    .filter(pl.col("Date") != "1/16/2023")
    .drop("Date")
)
stocks

AAPL,GOOG,IBM,MARA,NVDA
f64,f64,f64,f64,f64
5.257744,1.44037,-4.822947,-0.619579,14.159983
5.872672,4.478207,1.897465,-11.845387,3.609136
-2.258887,-9.846037,-0.978528,-16.265912,0.781988
1.174039,-0.284635,-0.42773,30.067568,0.57842
-3.828253,-5.539696,-3.295806,-13.376623,8.874133
2.944577,5.226636,-0.712268,-4.347826,2.59383
-1.675162,-3.201441,-3.232029,-16.45768,-3.871913
4.377104,12.581031,-1.402945,53.283302,12.018292
3.387098,3.513565,1.293556,-4.406365,4.097185


In [183]:
def calculate_covariance(df: pl.DataFrame):
    # Center columns with the mean
    centered_df = df.select(
        [(pl.col(c) - pl.col(c).mean()).alias(c) for c in df.columns]
    )

    # Calculate pairwise covariance and construct covariance matrix
    n = len(df) - 1
    cov_matrix = pl.DataFrame(
        [
            [(centered_df[c1] * centered_df[c2]).sum() / n for c1 in df.columns]
            for c2 in df.columns
        ],
        schema=df.schema,
    )

    return cov_matrix


n_stocks = len(stock_names)
stock_return = stocks.mean()
cov_mat = calculate_covariance(stocks)
cov_mat

AAPL,GOOG,IBM,MARA,NVDA
f64,f64,f64,f64,f64
8.891972,6.163829,2.499224,9.3736,9.096995
6.163829,19.498635,1.234442,17.023391,9.833599
2.499224,1.234442,6.123906,-1.107004,2.074088
9.3736,17.023391,-1.107004,292.463474,1.309068
9.096995,9.833599,2.074088,1.309068,36.943424


In [184]:
# Verifying covariance matrix calculation with Pandas calculation
stocks.to_pandas().cov()

,AAPL,GOOG,IBM,MARA,NVDA
AAPL,8.891972,6.163829,2.499224,9.373600,9.096995
GOOG,6.163829,19.498635,1.234442,17.023391,9.833599
IBM,2.499224,1.234442,6.123906,-1.107004,2.074088
MARA,9.373600,17.023391,-1.107004,292.463474,1.309068
NVDA,9.096995,9.833599,2.074088,1.309068,36.943424


# Optimization Model

In [185]:
cov_mat = cov_mat.to_pandas()
cov_mat.index = stock_names
m = Model("Portfolio")

vars = pd.Series(m.addVars(stock_names, lb=0), index=stock_names)
portfolio_risk = cov_mat.dot(vars).dot(vars)
m.setObjective(portfolio_risk, GRB.MINIMIZE)